In [1]:
import sys

sys.path.append("/Users/j/Documents/Forme/code/forme-groups")

from src.groups.group import Group


def main():
    # Create a default group
    group = Group(
        system_types_path="/Users/j/Documents/Forme/code/forme-groups/src/groups/system_types.json"
    )

    group.freeze_unit_types()

    print(group._group_unit_generator.unit_generator.unit_type_pool.get_type_aliases())

    print(group.group_units)
    
    unit1 = group.create_group_unit()

    group.create_group_unit(active_unit = unit1)

    print(group.group_units)

main()

['string', 'str', 'int', 'integer', 'float', 'bool', 'boolean', 'list', 'dict', 'dictionary', 'tuple', 'bytes']
[]
Creating new group unit with default nonce.


Exception: UnitTypePool must be frozen before generating units.